In [ ]:
# Import libs
import sys
import matplotlib.pyplot as plt
import matplotlib
from docplex.mp.model import Model
from docplex.util.environment import get_environment
from docplex.mp.relax_linear import LinearRelaxer
import matplotlib.patches as mpatches
import numpy as np

from datos import *

try:
    import docplex.mp
except:
    raise Exception('Please install docplex. See https://pypi.org/project/docplex/')

def imprimir_solucion(model):
    solution = model.solve()
    model.print_information()

    ancho = 100
    if solution:
        print(f"\n{'='*ancho}")
        print(f"{'RESULTADO'.center(ancho, ' ')}")
        print(f"{'='*ancho}")
        print(f"BENEFICIO: ${model.objective_value:.2f}")
        print(f"{'='*ancho}\n")

        # Imprimir variables
        print(f"\n{'Variables'.center(ancho, '-')}")
        print(f"{'Nombre'.ljust(40)}{'Valor'.rjust(30)}{'Reduced Cost'.rjust(30)}")
        print(f"{'-'*ancho}")
        for var in model.iter_variables():
            print(f"{var.name.ljust(40)}{str(round(var.solution_value, 2)).rjust(30)}{str(round(var.reduced_cost, 2)).rjust(30)}")
        
        # Imprimir restricciones
        print(f"{'Restricciones'.center(ancho, '-')}")
        print(f"{'Nombre'.ljust(40)}{'Slack'.rjust(30)}{'Dual Price'.rjust(30)}")
        print(f"{'-'*ancho}")
        for constraint in model.iter_constraints():
            print(f"{constraint.name.ljust(40)}{str(round(constraint.slack_value, 2)).rjust(30)}{str(round(constraint.dual_value, 2)).rjust(30)}")

        print(f"{'='*ancho}")
    else:
        print(f"{'='*ancho}")
        print("No se encontró una solución factible.")
        print(f"{'='*ancho}")

def imprimir_graficos(model, servicios, producido , costos, limites):
    # Datos necesarios
    produccion_graf = {servicio: producido[servicio].solution_value for servicio in servicios}
    total = sum(produccion_graf.values())

    fig, ax = plt.subplots()
    bars = ax.bar(produccion_graf.keys(), produccion_graf.values(), color='blue')
    # ax.axhline(y=total, color='r', linestyle='--', label='Total de producción')

    colores = {
        A: '#6b227d',
        B: '#ff6347',
        C: '#008000'
    }

    for servicio in servicios:
        ax.axhline(y=limites[servicio], color=colores[servicio], linestyle='--', label=f"Demanda {servicio}")
    ax.set_ylabel('Producción')
    ax.set_title('Producción con Límite Total')
    ax.legend()

    # Mostrar el valor sobre cada barra
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2.0, height, f'{height:.2f}', ha='center', va='bottom')

    costos = {
        "Produccion" : costos["materia_prima"].solution_value,
        "Translado" : costos["translado"].solution_value,
        "Horas Extra" : costos["horas_extra"].solution_value,
    }
    total = sum(costos.values())
    porcentaje_costos = {tipo: (valor / total) * 100 for tipo, valor in costos.items() if valor > 0}

    # Gráfico de pastel para representar el porcentaje de cada aceite en el producto final
    plt.figure(figsize=(8, 8))
    plt.pie(porcentaje_costos.values(), labels=porcentaje_costos.keys(), autopct='%1.1f%%', colors=['lightcoral', 'lightskyblue', 'lightgreen', 'gold', 'lightpink'])
    plt.title('Porcentaje de cada costo respecto del costo total')
    plt.show()

def perform_sensitivity_analysis(model):
    lp = LinearRelaxer.make_relaxed_model(model)
    lp.solve()
    cpx = lp.get_engine().get_cplex()
    return cpx.solution.sensitivity.rhs(), cpx.solution.sensitivity.objective()

def analisis_sensibilidad(model):
    ancho = 120
    print(f"\n{'='*ancho}")
    print(f"{'Análisis de sensibilidad'.center(ancho, ' ')}")
    print(f"{'='*ancho}")
    sensitivity_rhs, sensitivity_objective = perform_sensitivity_analysis(model)

    print(f"{'Variables'.center(ancho, '-')}")
    print(f"{'Variable'.ljust(50)}{'Coeficiente'.ljust(10)}{'Lim inferior'.rjust(30)}{'Lim superior'.rjust(30)}")
    print(f"{'-'*ancho}")

    for idx, var in enumerate(model.iter_variables()):
        coeficiente_actual = model.objective_coef(var)
        rango_inferior, rango_superior = sensitivity_objective[idx]
        if rango_inferior == -1e+20:
            rango_inferior = "Infinity"
        else:
            rango_inferior = str(round(rango_inferior, 2))
        if rango_superior == 1e+20:
            rango_superior = "Infinity"
        else:
            rango_superior = str(round(rango_superior, 2))
        
        print(f"{var.name.ljust(50)}{str(round(coeficiente_actual, 2)).ljust(10)}{rango_inferior.rjust(30)}{rango_superior.rjust(30)}")
    print(f"{'='*ancho}")

    print(f"{'Restricciones'.center(ancho, '-')}")
    print(f"{'Nombre'.ljust(40)}{'Lim inferior'.rjust(30)}{'Lim superior'.rjust(30)}")
    print(f"{'-'*ancho}")   
    for idx, c in enumerate(model.iter_constraints()):
        if idx < len(sensitivity_rhs):
            lim_inf, lim_sup = sensitivity_rhs[idx]
            if lim_inf == -1e+20:
                lim_inf = "Infinity"
            else:
                lim_inf = str(round(lim_inf, 2))
            if lim_sup == 1e+20:
                lim_sup = "Infinity"
            else:
                lim_sup = str(round(lim_sup, 2))
            print(f"{c.name.ljust(40)}{lim_inf.rjust(30)}{lim_sup.rjust(30)}")
    print(f"{'='*ancho}")

In [ ]:
real_rhs_value = 0
def graficar_marginal_disponibilidad(mdl, name_constraint):
    rhs_values = []
    dual_values = []                                              
    constraint_nameX = name_constraint # disponibilidad maquina hr/mes
    constraint_nameY = mdl.get_constraint_by_name(name_constraint) # Marginal Maquina 1
    m = 0.01

    def solve(mdl, c, rhs_value):
        print("---")
        print("- Adjusting RHS to: {0}".format(rhs_value))
        c.rhs = rhs_value
        solution = mdl.solve()
        
        if solution is not None:       
            print("* Production model solved with objective: {:g}".format(solution.objective_value))
            print("* Total benefit=%g" % solution.objective_value)
            for var in mdl.iter_variables():
                print(f"Production of {var.name}: {var.solution_value}")
            return solution
        else:
            print("No solution found for RHS value: {0}".format(rhs_value))
            return None  # Return None to indicate that the model is infeasible at this point

    # Report values for the chart

    def report(rhs_value, dual_value):
        rhs_values.append(rhs_value)
        dual_values.append(dual_value) #Diferencia con grafico Costo Op dem min B vs disp


    def plot(mdl):

        if not rhs_values or not dual_values:
            print("Error: `rhs_values` o `dual_values` están vacíos.")
            return

        if len(rhs_values) != len(dual_values):
            print("Error: `rhs_values` y `dual_values` deben tener la misma longitud.")
            return

        # Configuración de estilo
        matplotlib.rcParams.update({'font.size': 18})

        plt.figure(figsize=(12, 8))  # Tamaño de la gráfica

        # Dibujar líneas horizontales entre los puntos
        for i in range(len(rhs_values) - 1):
            plt.hlines(dual_values[i], rhs_values[i], rhs_values[i + 1], linewidth=4, color='C0')

        # Dibujar un vector con origen en el último punto hacia infinito horizontalmente
        plt.annotate('', xy=(plt.xlim()[1], dual_values[-1]), xytext=(rhs_values[-1], dual_values[-1]),
                    arrowprops=dict(arrowstyle="->", lw=2, color='C0'))

        # Dibujar la línea vertical para el RHS actual
        plt.axvline(x=real_rhs_value, color='g', linestyle='--', label='Valor actual')

        # Ajustar ticks de los ejes
        plt.xticks(rhs_values, rotation=45)  # Rotación para mayor legibilidad
        plt.yticks(dual_values)

        # Etiquetas y título
        plt.xlabel(f'{constraint_nameX}', labelpad=20, color='#DC143C')
        plt.ylabel(f'Valor Marginal\n{constraint_nameX}', rotation=0, labelpad=90, color='#DC143C')
        plt.title(f'Valor Marginal {constraint_nameX}', pad=30)

        # Configuración de rejilla
        plt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray', alpha=0.7)

        # Añadir leyenda
        plt.legend(['Valor actual', 'Segmentos duales'], loc='upper right', fontsize=12)

        # Mostrar la gráfica
        plt.tight_layout()
        plt.show()
        
    def perform_sensitivity_analysis2(mdl):
        lp = LinearRelaxer.make_relaxed_model(mdl)
        lp.solve()
        cpx = lp.get_engine().get_cplex()
        return cpx.solution.sensitivity.rhs()

    def iterate_over_rhs(mdl, constraint_nameX, constraint_nameY):
        c = mdl.get_constraint_by_name(constraint_nameX)
        if c is None:
            print("Constraint with name '{0}' not found.".format(constraint_nameX))
            return
                
        # Perform initial sensitivity analysis to get the starting lower and upper bounds
        initial_sensitivity = perform_sensitivity_analysis2(mdl)

        for c_sens, (lower, upper) in zip(mdl.iter_constraints(), initial_sensitivity):
            if c_sens.name == constraint_nameX:
                
                # Report the real RHS value for the chart
                global real_rhs_value #use the global keyword to declare that you want the function to update the global variable
                real_rhs_value = c_sens.rhs.constant
                print(f"c_sens: {c_sens}")
                print(f"real_rhs_value: {real_rhs_value}")
                report(real_rhs_value, constraint_nameY.dual_value)

            
                # *********Store and report the initial lower and upper bounds for the chart*********
                print("---Initial lower bound: {0}".format(lower))
                rhs = lower
                solution = solve(mdl, c, rhs)
                if solution is not None:
                    report(rhs, constraint_nameY.dual_value)

                print("---Initial upper bound: {0}".format(upper))
                rhs = upper
                solution = solve(mdl, c, rhs)
                if solution is not None:
                    report(rhs, constraint_nameY.dual_value)
                # ********* End of lower and upper bounds *********
                
                
                #Decrease rhs starting from lower bound - m
                rhs = lower - m
                while True:
                    if rhs < 0:
                        break ## Stop if the rhs is lower than 0                
                
                    solution = solve(mdl, c, rhs)                
                    if solution is None:
                        break  # Stop if the model is infeasible
                    else:
                        report(rhs, constraint_nameY.dual_value) #Diferencia con grafico funcional vs disp
                        
                    # Perform sensitivity analysis to get the new lower bound
                    new_sensitivity = perform_sensitivity_analysis2(mdl)                
                    for c_new_sens, (new_lower, _) in zip(mdl.iter_constraints(), new_sensitivity):
                        if c_new_sens.name == constraint_nameX: 
                            rhs = new_lower
                            if rhs < 0:
                                break ## Stop if the rhs is lower than 0                
                                
                            solution = solve(mdl, c, rhs)
                            if solution is None:
                                break  # Stop if the model is infeasible
                            report(c_new_sens.rhs.constant, constraint_nameY.dual_value)
                            
                            rhs = new_lower - m

                            break


                # Increase rhs starting from upper bound + m
                rhs = upper + m
                
                while True:
                    if rhs >= mdl.infinity:
                        break ## Stop if the rhs reaches or exceeds infinity

                    solution = solve(mdl, c, rhs)
                    if solution is None:
                        break  # Stop if the model is infeasible
                    else:
                        report(rhs, constraint_nameY.dual_value) #Diferencia con grafico funcional vs disp

                    # Perform sensitivity analysis to get the new upper bound
                    new_sensitivity = perform_sensitivity_analysis2(mdl)
                    for c_new_sens, (_, new_upper) in zip(mdl.iter_constraints(), new_sensitivity):
                        if c_new_sens.name == constraint_nameX:                        
                            rhs = new_upper
                            if rhs >= mdl.infinity:
                                break ## Stop if the rhs reaches or exceeds infinity

                            solution = solve(mdl, c, rhs)
                            if solution is None:
                                break  # Stop if the model is infeasible
                            report(c_new_sens.rhs.constant, constraint_nameY.dual_value)
                            
                            rhs = new_upper + m

                            break

    iterate_over_rhs(mdl, constraint_nameX, constraint_nameY)
    plot(mdl)

# PROBLEMA ORIGINAL - Papelera Godoy

### POSIBLES HIPOTESIS
1. si me llegan a entrar 7000 cajas, las cuales supera el limite de produccion, entonces no se produce nada, porque no cumple la restriccion -> eso significa que la empresa no trabaja / no toma el pedido ya que no tiene capacidad para producirlo

### Horas Extras:
Como para la produccion de B no tenemos restricciones, a cierto valor de demanda de B, se van a requerir horas extras 

In [ ]:
mdl = Model(name="TP: Equipo A SRL (CASO BASE)")

producido = dict()
cantidad_cajas = dict()
horas_produccion_producto = dict()

for servicio in servicios:
    producido[servicio] = mdl.continuous_var(name=f'Producido_{servicio}')
    cantidad_cajas[servicio] = mdl.continuous_var(name=f"Cantidad_Cajas_A_Comprar_Para_{servicio}")
    horas_produccion_producto[servicio] = mdl.continuous_var(name=f"Consumo_horas_produccion_de_{servicio}")
horas_maquina = mdl.continuous_var(name=f"Consumo_horas_maquina")
horas_extra = mdl.continuous_var(name=f"Horas_Extra")
capacidad_ociosa = mdl.continuous_var(name=f"Horas_Sobrantes")
cantidad_translados = mdl.continuous_var(name=f"Cantidad_Translados")

#-- RESTRICCIONES --
#-- DEMANDA MAXIMA --
for servicio in servicios:
    mdl.add_constraint(producido[servicio] <= LIMITES_PRODUCCION[servicio],f"Limite_de_produccion_{servicio}")

#-- CONSUMO RECURSOS --
# Consumo de cajas
for servicio in servicios:
    mdl.add_constraint(CAPACIDAD_CAJA[servicio]*cantidad_cajas[servicio] == producido[servicio],f"Capacidad_caja_{servicio}")

# Consumo de maquina
mdl.add_constraint(PRODUCCION_MAQUINA[M1]*horas_maquina == producido[A],f"Produccion_Maquina") # Solo el producto A tiene un proceso de maquina
mdl.add_constraint(horas_maquina <= LIMITE_HORAS_MAQUINA[M1],f"Limite_horas_maquina")

# Consumo horas trabajo
for servicio in servicios:
    mdl.add_constraint(PRODUCCION_HORA[servicio]*horas_produccion_producto[servicio] == producido[servicio],f"Produccion_hora_{servicio}")
horas_produccion = horas_produccion_producto[A] + horas_produccion_producto[B]
    
# Traslado
mdl.add_constraint(CAPACIDAD_TRANSLADO[A]*cantidad_translados == producido[A],f"Capacidad_traslado")

#-- Restricciones horarias --
mdl.add_constraint(PERSONAL_NECESARIO_MAQUINA[M1]*horas_maquina + horas_produccion == LIMITE_HORAS_HOMBRE + horas_extra - capacidad_ociosa,f"Restriccion_horaria")


#-- Max Horas extra --
mdl.add_constraint(horas_extra <= MAX_HORAS_EXTRA_PERSONAL, f"Maximo_horas_extra")

ingresos = mdl.sum(PRECIO_VENTA[servicio]*producido[servicio] for servicio in servicios)
costos_materia_prima = mdl.sum(PRECIO_CAJAS*cantidad_cajas[servicio] for servicio in servicios)
costo_horas_extra = horas_extra*PRECIO_HORA_EXTRA
costo_translado = cantidad_translados*PRECIO_TRANSLADO[A]

costos = costos_materia_prima + costo_horas_extra + costo_translado - COSTOS_FIJOS

beneficio = ingresos - costos
mdl.maximize(beneficio)

# Imprimir la soluciÃ³n
imprimir_solucion(mdl)
costos_totales = {
    "materia_prima": costos_materia_prima,
    "horas_extra": costo_horas_extra,
    "translado": costo_translado
}
analisis_sensibilidad(mdl)
imprimir_graficos(mdl, servicios, producido, costos_totales, LIMITES_PRODUCCION)
graficar_marginal_disponibilidad(mdl, "Limite_horas_maquina")
#graficar_marginal_disponibilidad(mdl,"Capacidad_traslado")

# PROBLEMA 2 (Agregar nuevo producto)

In [ ]:
# Servicios
servicios.append(C)
maquinas.append(M2)

# #-------- MODELO 2 --------

mdl = Model(name="TP: Equipo A SRL (NUEVO PRODUCTO)")

cantidad_tinta = mdl.continuous_var(name=f"Cantidad_tinta_a_comprar_para_diarios")
cantidad_cajas = dict()
producido = dict()
horas_produccion_producto = dict()
horas_maquina = dict()

for servicio in servicios:
    producido[servicio] = mdl.continuous_var(name=f'Producido_{servicio}')
    horas_produccion_producto[servicio] = mdl.continuous_var(name=f"Consumo_horas_produccion_de_{servicio}")
    if servicio != C:
        cantidad_cajas[servicio] = mdl.continuous_var(name=f"Cantidad_Cajas_A_Comprar_Para_{servicio}")

for maquina in maquinas:
    horas_maquina[maquina] = mdl.continuous_var(name=f"Consumo_horas_maquina__{maquina}")

horas_produccion = mdl.continuous_var(name=f"Consumo_horas_produccion")
horas_extra = mdl.continuous_var(name=f"Horas_Extra")
capacidad_ociosa = mdl.continuous_var(name=f"Horas_Sobrantes")
cantidad_translados_sobres = mdl.continuous_var(name=f"Cantidad_Translados_sobres")
cantidad_translados_diarios = mdl.continuous_var(name=f"Cantidad_Translados_diarios")

# #-- RESTRICCIONES --
# #-- DEMANDA MAXIMA --
for servicio in servicios:
    mdl.add_constraint(producido[servicio] <= LIMITES_PRODUCCION[servicio], f"Demanda_Maxima_{servicio}")

# #-- CONSUMO RECURSOS --
# # Consumo de cajas y tinta
for servicio in servicios:
    if servicio == C:
        mdl.add_constraint(cantidad_tinta == producido[servicio] * TINTA_DIARIO,f"Consumo_de_{servicio}")
    else: 
        mdl.add_constraint(CAPACIDAD_CAJA[servicio]*cantidad_cajas[servicio] == producido[servicio],f"Consumo_de_{servicio}")

# # Consumo de maquina
mdl.add_constraint(PRODUCCION_MAQUINA[M1]*horas_maquina[M1] == producido[A],f"Consumo_Maquina_1") 
mdl.add_constraint(PRODUCCION_MAQUINA[M2]*horas_maquina[M2] == producido[C], f"Consumo_Maquina_2")

for maquina in maquinas:
    mdl.add_constraint(horas_maquina[maquina] <= LIMITE_HORAS_MAQUINA[maquina], f"Limite_horas_{maquina}")

# Consumo horas trabajo
for servicio in servicios:
    mdl.add_constraint(PRODUCCION_HORA[servicio]*horas_produccion_producto[servicio] == producido[servicio],f"Consumo_horas_{servicio}")
horas_produccion = mdl.sum(horas_produccion_producto[servicio] for servicio in servicios)

# Translado
mdl.add_constraint(CAPACIDAD_TRANSLADO[A]*cantidad_translados_sobres == producido[A],f"Cantidad_traslados_sobres")
mdl.add_constraint(CAPACIDAD_TRANSLADO[C]*cantidad_translados_diarios == producido[C],f"Cantidad_traslados_diario")


#-- Restricciones horarias --
suma_horas_maquina = (PERSONAL_NECESARIO_MAQUINA[M1]*horas_maquina[M1]) + (PERSONAL_NECESARIO_MAQUINA[M2]*horas_maquina[M2])
mdl.add_constraint(suma_horas_maquina + horas_produccion == LIMITE_HORAS_HOMBRE + horas_extra - capacidad_ociosa,f"Restriccion_horaria")


#-- Max Horas extra --
mdl.add_constraint(horas_extra <= MAX_HORAS_EXTRA_PERSONAL,f"Maximo_horas_extra")

# #-- FUNCIONAL -- 
# # Ingresos
ingresos = mdl.sum(PRECIO_VENTA[servicio]*producido[servicio] for servicio in servicios)

# Costos
costo_materia_prima_sobres = mdl.sum(PRECIO_CAJAS*cantidad_cajas[servicio] for servicio in servicios if servicio != C) 
costo_materia_prima_diarios = PRECIO_TINTA * cantidad_tinta / 1000 
costos_materia_prima = costo_materia_prima_sobres + costo_materia_prima_diarios

costo_horas_extra = horas_extra*PRECIO_HORA_EXTRA

costo_translado = cantidad_translados_sobres*PRECIO_TRANSLADO[A] + cantidad_translados_diarios*PRECIO_TRANSLADO[C]

costos = costos_materia_prima + costo_horas_extra + costo_translado - COSTOS_FIJOS

beneficio = ingresos - costos
mdl.maximize(beneficio)

# Imprimir la soluciÃ³n
imprimir_solucion(mdl)
costos_totales = {
    "materia_prima": costos_materia_prima,
    "horas_extra": costo_horas_extra,
    "translado": costo_translado
}
analisis_sensibilidad(mdl)
imprimir_graficos(mdl, servicios, producido, costos_totales, LIMITES_PRODUCCION)

servicios.remove(C)
maquinas.remove(M2)

# PROBLEMA 3 (Control de calidad)

In [ ]:
# Reparacion

# --- Modelo 3

mdl = Model(name="TP: Equipo A SRL (CONTROL DE CALIDAD)")

producido = dict()
cantidad_cajas = dict()
horas_produccion_producto = dict()
for servicio in servicios:
    producido[servicio] = mdl.continuous_var(name=f'Producido_{servicio}')
    cantidad_cajas[servicio] = mdl.continuous_var(name=f"Cantidad_Cajas_A_Comprar_Para_{servicio}")
    horas_produccion_producto[servicio] = mdl.continuous_var(name=f"Consumo_horas_produccion_de_{servicio}")

horas_maquina = mdl.continuous_var(name=f"Consumo_horas_maquina")
horas_extra = mdl.continuous_var(name=f"Horas_Extra")
horas_reparacion_sobres = mdl.continuous_var(name=f"consumo_horas_reparacion_sobres")
capacidad_ociosa = mdl.continuous_var(name=f"Horas_Sobrantes")
cantidad_translados = mdl.continuous_var(name=f"Cantidad_Translados")
rechazados_por_maquina = mdl.continuous_var(name=f"cantidad_rechazados_maquina_1")

#-- RESTRICCIONES --
#-- DEMANDA MAXIMA --
for servicio in servicios:
    mdl.add_constraint(producido[servicio] <= LIMITES_PRODUCCION[servicio], f"Demanda_maxima_{servicio}")

#-- CONSUMO RECURSOS --
# Consumo de cajas
for servicio in servicios:
    mdl.add_constraint(CAPACIDAD_CAJA[servicio]*cantidad_cajas[servicio] == producido[servicio],f"Consumo_cajas_{servicio}")

# Consumo de maquina
mdl.add_constraint(PRODUCCION_MAQUINA[M1]*horas_maquina == producido[A], f"Consumo_maquina_1") # Solo el producto A tiene un proceso de maquina
mdl.add_constraint(horas_maquina <= LIMITE_HORAS_MAQUINA[M1],f"Limite_horas_maquina")
mdl.add_constraint(rechazados_por_maquina == producido[A] * PORCENTAJE_RECHAZO_MAQUINA[M1],f"Rechazados_maquina_1")

# Consumo horas trabajo
for servicio in servicios:
    mdl.add_constraint(PRODUCCION_HORA[servicio]*horas_produccion_producto[servicio] == producido[servicio],f"Consumo_horas_trabajo")
horas_produccion = mdl.sum(horas_produccion_producto[servicio] for servio in servicios)

# Translado
mdl.add_constraint(CAPACIDAD_TRANSLADO[A]*cantidad_translados == producido[A],f"Cantidad_traslado_{A}")

#-- Restricciones horarias --
mdl.add_constraint(PERSONAL_NECESARIO_MAQUINA[M1]*horas_maquina + horas_produccion + horas_reparacion_sobres == LIMITE_HORAS_HOMBRE + horas_extra - capacidad_ociosa,f"Restriccion_horaria")

#-- Rechazo maquinas --
mdl.add_constraint(rechazados_por_maquina == horas_reparacion_sobres * REPARACION_HORA[A],f"Rechazo_maquinas")

#-- Max Horas extra --
mdl.add_constraint(horas_extra <= MAX_HORAS_EXTRA_PERSONAL,f"Maximo_horas_extra")


ingresos = mdl.sum(PRECIO_VENTA[servicio]*producido[servicio] for servicio in servicios)

costos_materia_prima = mdl.sum(PRECIO_CAJAS*cantidad_cajas[servicio] for servicio in servicios)
costo_horas_extra = horas_extra*PRECIO_HORA_EXTRA
costo_translado = cantidad_translados*PRECIO_TRANSLADO[A]
costo_reparado = rechazados_por_maquina * COSTO_REPARACION_SOBRES

costos = costos_materia_prima + costo_horas_extra + costo_translado + costo_reparado

beneficio = ingresos - costos - COSTOS_FIJOS
mdl.maximize(beneficio)


# Imprimir la soluciÃ³n
imprimir_solucion(mdl)
costos_totales = {
    "materia_prima": costos_materia_prima,
    "horas_extra": costo_horas_extra,
    "translado": costo_translado
}
analisis_sensibilidad(mdl)
imprimir_graficos(mdl, servicios, producido, costos_totales, LIMITES_PRODUCCION)